In [ ]:
import toml
import re
import urllib
import benchlingapi
import requests_html
from paulssonlab.api import *

In [ ]:
%load_ext autoreload
%autoreload 2

# Code

# Setup

In [ ]:
config = toml.load("config.toml")

In [ ]:
session = benchlingapi.Session(config["benchling"]["api_key"])

# Reference

In [ ]:
folder = get_folder(session, project="Test")

In [ ]:
dna = session.DNASequence.find_by_name("V37m", folder_id=folder.id)

In [ ]:
# gb = get_genbank(addgene_sequences('http://www.addgene.org/121010/')['depositor_full'][0])[0]
gb = get_genbank(
    addgene_sequences("https://www.addgene.org/108537/")["addgene_full"][0]
)[0]

In [ ]:
dna = genbank_to_benchling(session, gb, "seq3", folder.id, accession="addgene_blah")
dna.save()